In [6]:
import os
import sys
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, parent_dir)



from Camera import Camera
from Frame import Frame
import Plotters
import Utils as Utils

from GaussianSplat import GaussianSplat


import numpy as np
from plyfile import PlyData
import numpy as np
import argparse
from io import BytesIO
import matplotlib.pyplot as plt
import Plotters
import plotly
import plotly.graph_objects as go
import scipy.io
import pandas as pd
from Render import Render

import pickle
import numpy as np
import math

import matplotlib.pyplot as plt
import Utils
%matplotlib qt


path = 'C:/Users/Roni/Documents/gs_input/frames_model.pkl'
dict_path  = 'G:/My Drive/Research/gaussian_splatting/gaussian_splatting_input/mov30_2024_11_12_darkan/frames_model.pkl'

# dict_path = 'D:/Documents/data_for_gs/fly_gray/dict/frames_model.pkl'
image_path = 'D:/Documents/data_for_gs/fly_gray/'
frame = 1400
with open(dict_path,'rb') as f:
    frames = pickle.load(f)

frames_per_cam = [Frame(image_path,frame,cam_num, frames_dict = frames) for cam_num in range(4)]


In [72]:

file_name = 'dens_intv_100_densify_grad_0.0002_scaling_0.01'
input_file = f"D:/Documents/gaussian_model_output/hull_2dgs_2/900/{file_name}/point_cloud/iteration_20000/point_cloud.ply"

image_size = frames_per_cam[0].image_size
gs = GaussianSplat(input_file,image_size = frames_per_cam[0].image_size)  
cam = frames_per_cam[0]
gs.save_gs(name = '_2dsplat')
# render = Render(gs,cam,tiles = [1,10],block_xy = [16,16], image_size = [160,160],gaus3d = False,filtersze = 0.707)

# image,depth = render.render_image()
# plt.figure(),plt.imshow(image)
# plt.figure(),plt.imshow(depth/np.max(depth))
# file_name = 'gradth_0.0002_dense_until_15000_densify_from_500' 2dgs


FileNotFoundError: [Errno 2] No such file or directory: 'D:/Documents/gaussian_model_output/hull_2dgs_2/900/dens_intv_100_densify_grad_0.0002_scaling_0.01/point_cloud/iteration_20000/point_cloud.ply'

In [76]:
# file_name = 'gradth_0.0002_dense_until_15000_densify_from_500' 2dgs
file_name = 'dens_intv_1000_densify_grad_0.0002_scaling_0.01'
# file_name = 'gradth_0.0002_dense_until_15000_densify_from_1000' #2dgs

input_file = f"D:/Documents/gaussian_model_output/hull_3dgs/1400/{file_name}/point_cloud/iteration_5000/point_cloud.ply"
image_size = frames_per_cam[0].image_size

gs = GaussianSplat(input_file,image_size = image_size)  

vertices = PlyData.read(input_file)["vertex"]
vertices = gs.xyz


FileNotFoundError: [Errno 2] No such file or directory: 'D:/Documents/gaussian_model_output/hull_3dgs/1400/dens_intv_1000_densify_grad_0.0002_scaling_0.01/point_cloud/iteration_5000/point_cloud.ply'

In [74]:
import sh_utils
import plotly.graph_objects as go
import plotly.io as pio

gs = GaussianSplat(input_file,image_size = image_size)  

vertices = PlyData.read(input_file)["vertex"]
vertices = gs.xyz

sh = gs.sh
rgb = gs.color
xyz = gs.xyz
opacity = gs.opacity

grayscale = rgb[:,0]
grayscale = (grayscale - grayscale.min()) / (grayscale.max() - grayscale.min())

xyz = xyz[grayscale <1,:]
grayscale = grayscale[grayscale <1]


fig = go.Figure()
color_pts = ['red','green','blue','black']

im_name = list(frames.keys())[0]
fig.add_trace(go.Scatter3d(
    x=xyz[:,0],
    y=xyz[:,1],
    z=xyz[:,2],
    mode='markers',
    marker=dict(
        size=3,
        color=grayscale,  # Color for each point
        colorscale='Gray',  # Choose a colorscale
        opacity=0.5,  # Use the opacity per point
    ),
    hovertemplate='Color: %{marker.color:.4f}<extra></extra>',  # Show color as float
))


# Save the figure as an HTML file
output_html = f"G:/My Drive/Research/gaussian_splatting/gaussian_splatting_output/fly_gaussian/sagie/html_files/frame_{frame}.html"
# pio.write_html(fig, file=output_html, auto_open=False)
fig.update_layout(scene=dict(aspectratio=dict(x=1, y=1, z=0.8),aspectmode = 'data',
                            camera_eye=dict(x=1.2, y=1.2, z=0.6)))
fig.show()


In [69]:
# rotate to easy wand and project to 2d
import matplotlib.pyplot as plt

projeced = frames_per_cam[0].project_with_proj_mat(xyz)
fig,ax = plt.subplots(2,2)
pt_cloud = xyz.copy()
for idx in range(4):
    vertices_homo = frames_per_cam[idx].homogenize_coordinate(pt_cloud)
    points2d = frames_per_cam[idx].project_on_image(vertices_homo)
    ax[idx//2,np.mod(idx,2)].imshow(frames_per_cam[idx].im, cmap = 'gray')
    ax[idx//2,np.mod(idx,2)].scatter(points2d[:,0],points2d[:,1],color = 'red', alpha = 0.2, s= 3,cmap = 'gray')

    pt_cloud = frames_per_cam[idx].intersection_per_cam(pt_cloud)


C:\Users\Roni\AppData\Local\Temp\ipykernel_20024\2925251670.py:11: UserWarning:

No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored



In [70]:
import matplotlib.pyplot as plt

fig,ax = plt.subplots(2,2)

for idx in range(4):
    vertices_homo = frames_per_cam[idx].homogenize_coordinate(pt_cloud)
    points2d = frames_per_cam[idx].project_on_image(vertices_homo)
    ax[idx//2,np.mod(idx,2)].imshow(frames_per_cam[idx].im, cmap = 'gray')
    ax[idx//2,np.mod(idx,2)].scatter(points2d[:,0],points2d[:,1],color = 'red', alpha = 0.2, s= 3,cmap = 'gray')



C:\Users\Roni\AppData\Local\Temp\ipykernel_20024\40695633.py:9: UserWarning:

No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored



In [71]:
import sh_utils
import plotly.graph_objects as go
import plotly.io as pio



fig = go.Figure()
color_pts = ['red','green','blue','black']

im_name = list(frames.keys())[0]
fig.add_trace(go.Scatter3d(
    x=pt_cloud[:,0],
    y=pt_cloud[:,1],
    z=pt_cloud[:,2],
    mode='markers',
    marker=dict(
        size=3,
        color=grayscale,  # Color for each point
        colorscale='Gray',  # Choose a colorscale
        opacity=0.5,  # Use the opacity per point
    ),
    hovertemplate='Color: %{marker.color:.4f}<extra></extra>',  # Show color as float
))


# Save the figure as an HTML file
output_html = f"G:/My Drive/Research/gaussian_splatting/gaussian_splatting_output/fly_gaussian/sagie/html_files/frame_{frame}.html"
# pio.write_html(fig, file=output_html, auto_open=False)
fig.update_layout(scene=dict(aspectratio=dict(x=1, y=1, z=0.8),aspectmode = 'data',
                            camera_eye=dict(x=1.2, y=1.2, z=0.6)))
fig.show()



In [20]:
vertices
frame_name = 'P900CAM2.jpg'
vertices_homo = frames[frame_name].homogenize_coordinate(vertices)
points2d = frames[frame_name].project_on_image(vertices_homo.T)
plt.figure()
plt.imshow(frames[frame_name].image)
plt.scatter(points2d[:,0],points2d[:,1],s = 3,color = 'red')

In [10]:
import sh_utils
import plotly.graph_objects as go
import plotly.io as pio
cam = 0
im_name = list(frames.keys())[-1]

image = f'{im_name.split("CAM")[0]}CAM{cam+1}.jpg'
fig = go.Figure()
color_pts = ['red','green','blue','black']

for cam in range(4):
    image = f'{im_name.split("CAM")[0]}CAM{cam+1}.jpg'
    Plotters.scatter3d(fig,frames[image].voxels_with_idx,color_pts[cam],3,f'camera {cam+1}',show_colorbar = False)
# Save the figure as an HTML file
output_html = f"G:/My Drive/Research/gaussian_splatting/gaussian_splatting_output/fly_gaussian/dataset and current outputs/2d_splatting/html_files/initial_points_1419_zbuffer_per_view.html"

pio.write_html(fig, file=output_html, auto_open=False)
fig.show()


cam = 0
im_name = list(frames.keys())[0]

image = f'{im_name.split("CAM")[0]}CAM{cam+1}.jpg'
fig = go.Figure()
color_pts = ['red','green','blue','black']

image = f'{im_name.split("CAM")[0]}CAM{cam+1}.jpg'
Plotters.scatter3d(fig,frames[image].points_in_ew_frame_homo,color_pts[cam],3,f'camera {cam+1}',show_colorbar = False)
# Save the figure as an HTML file
output_html = f"G:/My Drive/Research/gaussian_splatting/gaussian_splatting_output/fly_gaussian/dataset and current outputs/2d_splatting/html_files/initial_points_1419_full_hull.html"
pio.write_html(fig, file=output_html, auto_open=False)
fig.show()


In [4]:
cam = frames[f'{image_names[2]}.jpg'].get_croped_camera()
render = Render(gs,cam,tiles = [1,10],block_xy = [16,16], image_size = [160,160],gaus3d = False,filtersze = 0.707)

image = render.render_image()
plt.figure(),plt.imshow(image)

(<Figure size 640x480 with 1 Axes>,
 <matplotlib.image.AxesImage at 0x1c19e8a32b0>)

In [24]:
tiled_axes = np.tile(gs.radius,(2,1)).T
radii = np.hstack([tiled_axes,gs.radius[:,np.newaxis]*0 + 0.1])
# Example ellipsoid parameters
centers = gs.xyz[radii[:,0]<10]  # Centers of ellipsoids

rotations = gs.rotation[radii[:,0]<10]  # Rotation matrices (identity here)
sum(radii[:,0]<7)

175

In [35]:
xyz_points = np.column_stack([vertices['x'],vertices['y'],vertices['z']])
grid_shape

array([1, 1, 1])

In [12]:
import open3d as o3d
from skimage import measure   
from plyfile import PlyData
import numpy as np

input_file = "G:/My Drive/Research/gaussian_splatting/gaussian_splatting_output/fly_gaussian/output/iterations_5000_densification_interval_700_ld_5_densify_until4000_ln05_lr001_ori_100_dr0/1419/point_cloud/iteration_5000/point_cloud.ply"


vertices = PlyData.read(input_file)["vertex"]

vertices = PlyData.read(input_file)["vertex"]
xyz_points = np.column_stack([vertices['x'],vertices['y'],vertices['z']])
# pcd = o3d.geometry.PointCloud()
# pcd.points = o3d.utility.Vector3dVector(xyz_points)


voxel_size = 0.0001

# Step 2: Define voxel size and grid dimensions
# voxel_size = 0.5  # Adjust based on your needs
min_bound = xyz_points.min(axis=0)  # Minimum coordinates (x_min, y_min, z_min)
max_bound = xyz_points.max(axis=0)  # Maximum coordinates (x_max, y_max, z_max)
grid_shape = np.ceil((max_bound - min_bound) / voxel_size).astype(int)  # Grid shape

# Step 3: Initialize a 3D NumPy array (voxel grid)
voxel_grid = np.zeros(grid_shape, dtype=np.int8)

# Step 4: Map points to voxel indices
indices = np.floor((xyz_points - min_bound) / voxel_size).astype(int)

# Step 5: Populate the voxel grid
for idx in indices:
    voxel_grid[tuple(idx)] = 1  # Mark as occupied



# Step 2: Create the voxel grid
# voxel_size = 0.00002  # Adjust this to change voxel size
# voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd, voxel_size=voxel_size)
# o3d.visualization.draw_geometries([voxel_grid])
vertices, faces,normals, values = measure.marching_cubes(voxel_grid,0,allow_degenerate=False,spacing=(1, 1, 1))  



mesh = o3d.geometry.TriangleMesh()
vertices = vertices.astype(int)



# Assign vertices, faces, and normals to the mesh
mesh.vertices = o3d.utility.Vector3dVector(xyz_points)
mesh.triangles = o3d.utility.Vector3iVector(faces)
# mesh.normals = normals

# mesh.compute_vertex_normals()
# vertex_normals = -np.asarray(mesh.vertex_normals)


o3d.visualization.draw_geometries([mesh], point_show_normal=True, mesh_show_wireframe=True)
# o3d.io.write_triangle_mesh(f'D:/Documents/gs_output/test.ply', mesh)

In [13]:
vertices = PlyData.read(input_file)["vertex"]
xyz_points = np.column_stack([vertices['x'],vertices['y'],vertices['z']])
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz_points)

voxel_size = 0.00002  # Adjust this to change voxel size
voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd, voxel_size=voxel_size)
o3d.visualization.draw_geometries([voxel_grid])

In [ ]:
from scipy import io
import numpy as np
import open3d as o3d
from skimage import measure   
rotate_to_lab = False

def get_3d_voxel_grid(indices):
    volume_shape = (max(indices[:,0])+2, max(indices[:,1])+2, max(indices[:,2])+2)
    voxels = np.zeros(volume_shape)
    for ix,iy,iz in indices:
        voxels[ix-1, iy-1, iz-1] = 1
    return voxels
    

# load body and wing data to dictionary
name_of_parts = ['body','wing_l','wing_r']
fly_hull = {name_of_part : data_dict[files_to_load[0]].query(f'{name_of_part} == 1')[['x','y','z']].to_numpy() for name_of_part in name_of_parts}
pcd = {name_of_part : o3d.geometry.PointCloud(o3d.utility.Vector3dVector(fly_hull[name_of_part])) for name_of_part in name_of_parts}
voxel_grid = get_3d_voxel_grid(np.vstack((fly_hull.values())))



voxel_grid

vertices, faces,normals, values = measure.marching_cubes(voxel_grid,0,allow_degenerate=False,spacing=(1, 1, 1))  


mesh = o3d.geometry.TriangleMesh()
vertices = vertices.astype(int)
if rotate_to_lab == True:
    real_hull_in_vertex_order =  np.dot(rot_mat,np.vstack([real_coord[ax][vertices[:,idx]]] for idx,ax in enumerate(['x','y','z']))).T 
else:
    real_hull_in_vertex_order =  np.vstack([real_coord[ax][vertices[:,idx]]] for idx,ax in enumerate(['x','y','z'])).T 


# Assign vertices, faces, and normals to the mesh
mesh.vertices = o3d.utility.Vector3dVector(real_hull_in_vertex_order)
mesh.triangles = o3d.utility.Vector3iVector(faces)
mesh.compute_vertex_normals()
vertex_normals = -np.asarray(mesh.vertex_normals)


o3d.visualization.draw_geometries([mesh], point_show_normal=True, mesh_show_wireframe=True)
o3d.io.write_triangle_mesh(f'{path}/mov{mov}_frame{frame}.ply', mesh)

In [13]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def plot_ellipse_in_3d(center, radii, rotation_matrix, ax, num_points=100):
    """
    Plots a single 2D ellipse in 3D space.
    
    Args:
        center (array-like): The 3D coordinates of the ellipse center (x, y, z).
        radii (array-like): The radii of the ellipse (r_x, r_y).
        rotation_matrix (ndarray): A 3x3 rotation matrix for the ellipse orientation.
        ax (Axes3D): A Matplotlib 3D axis object.
        num_points (int): Number of points to represent the ellipse.
    """
    # Create a 2D ellipse in its local coordinate system
    theta = np.linspace(0, 2 * np.pi, num_points)
    ellipse_2d = np.array([radii[0] * np.cos(theta), radii[1] * np.sin(theta), np.zeros_like(theta)])
    
    # Rotate the ellipse to its orientation in 3D space
    ellipse_3d = rotation_matrix @ ellipse_2d
    
    # Translate the ellipse to its center in 3D space
    ellipse_3d += np.array(center).reshape(3, 1)
    
    # Plot the ellipse
    ax.plot(ellipse_3d[0, :], ellipse_3d[1, :], ellipse_3d[2, :])

# Define your data: centers, radii, and rotation matrices
# Example ellipsoid parameters
tiled_axes = np.tile(gs.radius,(2,1)).T
radii = gs.axes
radius = gs.radius
# Example ellipsoid parameters
centers = gs.xyz[radius<10]  # Centers of ellipsoids
rotation_matrices = gs.rotation[radius<10]  # Rotation matrices (identity here)
radii = radii[radius<10]

# Create a 3D plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Plot each ellipse
for center, radii, rotation_matrix in zip(centers, radii, rotation_matrices):
    plot_ellipse_in_3d(center, radii, rotation_matrix, ax)

# Set labels and aspect
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
ax.set_box_aspect([1, 1, 1])  # Equal aspect ratio

plt.show()


In [22]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def plot_colored_ellipse_in_3d(center, radii, rotation_matrix, ax, color='blue', alpha=0.5, num_points=100):
    """
    Plots a single 2D ellipse in 3D space with color and opacity.
    
    Args:
        center (array-like): The 3D coordinates of the ellipse center (x, y, z).
        radii (array-like): The radii of the ellipse (r_x, r_y).
        rotation_matrix (ndarray): A 3x3 rotation matrix for the ellipse orientation.
        ax (Axes3D): A Matplotlib 3D axis object.
        color (str): Color of the ellipse.
        alpha (float): Opacity of the ellipse (0 is fully transparent, 1 is fully opaque).
        num_points (int): Number of points to represent the ellipse.
    """
    # Create a 2D ellipse in its local coordinate system
    theta = np.linspace(0, 2 * np.pi, num_points)
    ellipse_2d = np.array([radii[0] * np.cos(theta), radii[1] * np.sin(theta), np.zeros_like(theta)])
    
    # Rotate the ellipse to its orientation in 3D space
    ellipse_3d = rotation_matrix @ ellipse_2d
    
    # Translate the ellipse to its center in 3D space
    ellipse_3d += np.array(center).reshape(3, 1)
    
    # Generate a mesh for the ellipse
    X, Y, Z = ellipse_3d[0], ellipse_3d[1], ellipse_3d[2]
    
    # Triangulate for surface plotting
    ax.plot_trisurf(X, Y, Z, color=color, alpha=alpha)

# Define your data: centers, radii, and rotation matrices
idx = (gs.opacity < 1) & (gs.opacity > 0) & (gs.color[:,0] > 0) & (gs.color[:,1] > 0) & (gs.color[:,2] > 0)
centers = gs.xyz[idx]  # Centers of ellipsoids
rotation_matrices = gs.rotation[idx]  # Rotation matrices
radii = gs.axes[idx]  # Radii

# Create a 3D plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Colors for each ellipse (use a colormap or specify colors directly)
colors = gs.color[idx]
alpha = gs.opacity[idx]

# Plot each ellipse
for center, radii, rotation_matrix, color, opa in zip(centers, radii, rotation_matrices, colors,alpha):
    plot_colored_ellipse_in_3d(center, radii, rotation_matrix, ax, color=color, alpha=opa)

# Set labels and aspect
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
ax.set_box_aspect([1, 1, 1])  # Equal aspect ratio

plt.show()


In [20]:
color

array([-0.0027495 , -0.0027495 , -0.00275007])

In [15]:
import numpy as np
import plotly.graph_objects as go

def generate_ellipsoid(center, radii, rotation_matrix, u_samples=50, v_samples=50):
    u = np.linspace(0, 2 * np.pi, u_samples)
    v = np.linspace(0, np.pi, v_samples)
    x = radii[0] * np.outer(np.cos(u), np.sin(v))
    y = radii[1] * np.outer(np.sin(u), np.sin(v))
    z = radii[2] * np.outer(np.ones_like(u), np.cos(v))
    
    # Apply rotation and translation
    points = np.dot(rotation_matrix, np.array([x.ravel(), y.ravel(), z.ravel()]))
    x, y, z = points[0].reshape(x.shape), points[1].reshape(y.shape), points[2].reshape(z.shape)
    
    return x + center[0], y + center[1], z + center[2]
# Example ellipsoid parameters
tiled_axes = np.tile(gs.radius,(2,1)).T
radii = np.hstack([tiled_axes,gs.radius[:,np.newaxis]*0 + 0.1])
# Example ellipsoid parameters
centers = gs.xyz[radii[:,0]<7]  # Centers of ellipsoids
rotations = gs.rotation[radii[:,0]<7]  # Rotation matrices (identity here)
radii = radii[radii[:,0]<7]

fig = go.Figure()

for center, radius, rotation in zip(centers, radii, rotations):
    # Generate ellipsoid surface
    x, y, z = generate_ellipsoid(center, radius, rotation)
    fig.add_trace(go.Surface(x=x, y=y, z=z, opacity=0.5, colorscale='Viridis'))
    
    # Add principal axes
    for i in range(3):
        axis_start = center - radius[i] * rotation[:, i]
        axis_end = center + radius[i] * rotation[:, i]
        fig.add_trace(go.Scatter3d(
            x=[axis_start[0], axis_end[0]],
            y=[axis_start[1], axis_end[1]],
            z=[axis_start[2], axis_end[2]],
            mode='lines',
            line=dict(color='red', width=4)
        ))

# Set layout
fig.update_layout(scene=dict(
    xaxis_title='X',
    yaxis_title='Y',
    zaxis_title='Z'
))

fig.show()


In [5]:
plt.figure(),plt.imshow(render.distortion)
plt.figure(),plt.imshow(render.depth)
plt.figure(),plt.imshow(render.median_depth)
plt.figure(),plt.imshow(render.alpha_map,'gray')

shape = render.normal_map.shape

image_reshaped = render.normal_map.reshape(-1, 3)  # Flatten spatial dimensions
transformed = (cam.world_to_cam[:,:3] @ image_reshaped.T).T 

image_transformed = transformed.reshape(shape[0], shape[1], 3)
plt.figure()
plt.imshow(image_transformed[:,:,:]*0.5 + 0.5)

gs.scale

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


array([[1.31455401e-04, 2.04177949e-05, 1.60000000e-10],
       [1.43049244e-04, 2.86818056e-05, 1.60000000e-10],
       [5.52020167e-05, 3.68848277e-05, 1.60000000e-10],
       ...,
       [5.93517494e-05, 1.06731518e-04, 1.60000000e-10],
       [4.35989386e-05, 7.56119080e-05, 1.60000000e-10],
       [8.90756940e-05, 2.15409881e-05, 1.60000000e-10]])

In [6]:
plt.figure(),plt.imshow(render.depth/render.alpha_map)
depth_expected = render.depth/render.alpha_map
depth_ratio = 1
surf_depth = depth_expected*(1-depth_ratio) + (depth_ratio) * render.median_depth
surf_depth[np.isnan(surf_depth)] = 0

depth = surf_depth / surf_depth.max()
plt.imshow(depth, cmap='turbo')

C:\Users\Roni\AppData\Local\Temp\ipykernel_25240\2664146519.py:1: RuntimeWarning:

invalid value encountered in divide

C:\Users\Roni\AppData\Local\Temp\ipykernel_25240\2664146519.py:2: RuntimeWarning:

invalid value encountered in divide



In [ ]:
plt.imshow(depth, cmap='turbo', vmin=depth.min(), vmax=depth.max())
plt.colorbar()
plt.title('Depth Map with Turbo Colormap')
plt.show()


In [34]:
# render image 
image_name = f'{image_names[0]}.jpg'

cam1 = frames[image_name].get_croped_camera()
cam4 = frames[f'{image_names[0]}.jpg'].get_croped_camera()

rotmat = np.array([[1,0,0],[0,1,0],[0,0,1]])

K_crop = cam4.K
K_crop[0,0] = K_crop[0,0]
K_crop[1,1] = K_crop[1,1]

R = rotmat @ cam4.R
X0 = cam4.X0 + np.array([[0,0,0]]).T
cam = Camera(cam1.path,5,cam = {'camera': np.hstack((K_crop,R,X0))[:, :, np.newaxis]},image_size = [160,160])

cam = frames[f'{image_names[0]}.jpg'].get_croped_camera()



render = Render(gs,cam,tiles = [1,10],block_xy = [16,16], image_size = [160,160])
image = render.render_image()
plt.figure(),plt.imshow(image)
# plt.figure(),plt.imshow(render.depth)
np.unique(render.depth)


UnboundLocalError: local variable 'm' referenced before assignment

In [7]:
gs.scale

array([[8.06498141e-05, 7.69689944e-05, 1.60000000e-06],
       [7.98608723e-05, 8.76289931e-05, 1.60000000e-06],
       [8.31628697e-05, 9.40807091e-05, 1.60000000e-06],
       ...,
       [9.92269857e-05, 8.62566106e-05, 1.60000000e-06],
       [8.34508671e-05, 5.89549979e-05, 1.60000000e-06],
       [7.89204542e-05, 8.14726523e-05, 1.60000000e-06]])

In [7]:
cam = frames[f'{image_names[0]}.jpg'].get_croped_camera()
render = Render(gs,cam,tiles = [1,10],block_xy = [16,16], image_size = [160,160],T = T,center = center,gaus3d = True)

image = render.render_image()
plt.figure(),plt.imshow(image)
# plt.figure(),plt.imshow(render.depth)
np.unique(render.depth)


NameError: name 'T' is not defined

In [71]:
# projection
xyz = np.column_stack((vertices["x"], vertices["y"], vertices["z"]))

im_name= 'P1408CAM1.jpg'
fig,axs = plt.subplots(2,2)
for cam in range(4):
    cam_obj  = frames[f'{image_names[cam]}.jpg'].get_croped_camera()

    image = f'{im_name.split("CAM")[0]}CAM{cam+1}.jpg'
    homo_voxels_with_idx = frames[image].add_homo_coords(xyz)
    proj = cam_obj.project_with_proj_mat(xyz)
    axs[cam // 2,cam % 2].imshow(frames[image].croped_image)
    axs[cam // 2,cam % 2].scatter(proj[:,0],proj[:,1],s = 3,c = rgb)

## Elliptical Gaussian (from EWA article) - Math explained


we define an elliptical gaussian: 

Equation 1: $G_\mathbf{V}(\mathbf{x} - \mathbf{p}) = \frac{1}{2 \pi |\mathbf{V}|} \frac{1}{2} e^{-\frac{1}{2} (\mathbf{x} - \mathbf{p})^T \mathbf{V}^{-1} (\mathbf{x} - \mathbf{p})}.$

* $V$ is the covariance matrix
* $\mathbf{x}$ - a 3d coordinate (in gaussian axes)
* $\mathbf{p}$ - the mean of the gaussian 
### Affine projection of the gauusian
We define an arbitrary affine mapping from object space $ \mathbf{u} = \Phi(\mathbf{x}) $, where $ \Phi(\mathbf{x}) = \mathbf{Mx + c} $, 

and we substitute $ \mathbf{x} = \Phi^{-1}(\mathbf{u}) $. Since $ \Phi^{-1}(\mathbf{u}) = \mathbf{M}^{-1}(\mathbf{u} - \mathbf{c}) $, we get:

$$
G_\mathbf{V}(\Phi^{-1}(\mathbf{u}) - \mathbf{p}) = 
\frac{1}{|M^{-1}|} \cdot \frac{1}{(2 \pi)^{3/2} |\mathbf{V}|^{1/2}} e^{-\frac{1}{2} (\mathbf{M}^{-1}(\mathbf{u} - \mathbf{c}) - \mathbf{p})^T \mathbf{V}^{-1} (\mathbf{M}^{-1}(\mathbf{u} - \mathbf{c}) - \mathbf{p})} = 
$$

$$
\frac{1}{|M^{-1}|} \cdot \frac{1}{(2 \pi)^{3/2} |\mathbf{V}|^{1/2}} e^{-\frac{1}{2} (\mathbf{u} - \mathbf{c} - \mathbf{Mp})^T \mathbf{M}^{-1} \mathbf{V}^{-1} (\mathbf{M}^{-1})^T (\mathbf{u} - \mathbf{c} - \mathbf{Mp})} = 
$$

$$
\frac{1}{|M^{-1}|} \cdot \frac{1}{(2 \pi)^{3/2} |\mathbf{V}|^{1/2}} e^{-\frac{1}{2} (\mathbf{u} - \Phi(\mathbf{p}))^T (\mathbf{MVM}^T)^{-1} (\mathbf{u} - \Phi(\mathbf{p}))}.
$$

from this, to ease the writing we define equation 2: 

Equation 2: $G \mathbf{V}(\Phi^{-1}(u) - \mathbf{p}) = \frac{1}{|\mathbf{M}^{-1}|} G_{ \mathbf{M} \mathbf{V} \mathbf{M}^T} (u - \Phi(\mathbf{p})).$

#### The covariance matrix
$V$, the covariance matrix is a symetrix matrix that can be ragarded as a transformation. it scales and shears the data (thus defining the orientation and ratio between the gaussian principal axes). 
As mentioned above, the gaussian is transformed using an affine mapping $\Phi(X)$. Usually we use this kind of mapping to transform a vector or a 3D point to a new FoR (change basis). Because the covariance matrix is a transformation, in order to change its FoR (basis), we will perform $\mathbf{V_{1}} = MVM^{T}$. We first multiply $VM^{T}$  - transform (whatever xyz we multiply) from camera to body ($M^{T}$) and then multiply by $V$ which is in object coordinates - now we have coordinates in object basis after shear and scale. we then multiply by $M$ again to transform back to camera coordinates. 


### Projection


![alt text](ray_ewa.png)
#### Ray space

We converted to camera space, now we want to convert to ray space. The coordinates in ray space are defined such that $x_0, x_1$ are the coordinates on the image plane and the devision by $u_2$ adds prespective. $u_3$ is then defined as the euclidean distance from the point to the origin (camera). 


$$ \text{ray space to camera: }
\begin{pmatrix}
x_0 \\
x_1 \\
x_2 \\

\end{pmatrix} = m(u) = \begin{pmatrix}
u_0 / u_2 \\
u_1/u_2 \\
||(u_0, u_1, u_2)||^{T} \\
\end{pmatrix}

\text{,camera to ray space: }
\begin{pmatrix}
u_0 \\
u_1 \\
u_2 \\

\end{pmatrix} = m^{-1}(x) = \begin{pmatrix}
x_0/||(x_0,x_1,1)^T|| \cdot x_2 \\
x_1/||(x_0,x_1,1)^T|| \cdot x_2 \\
1/||(x_0,x_1,1)^T|| \cdot x_2 \\

\end{pmatrix}


$$
  
Ray space is defined with 3 coordinates $(x_0,x_1,x_2)$ where $x_0$ and $x_1$ represents the location of the pixel and $x_3$ is the euclidean distance from the camera origin to $(x_1,x_2). This transformation is not affine due to the devision by $u_2$, parallel lines are not preserved and the shape of the gaussian will be distorted. (in addition, the dimention is problematic).
In order to solve this a local affine approximation $\mathbf{m_{u_k}}$ of the projection transformation is defined. The approximation is the first to terms of the Tayor expansion of $\mathbf{m}$ at point $\mathbf{u}_{k}$:
$$\mathbf{m_{u_k}\mathbf(u)} = \mathbf x_k+\mathbf J_{\mathbf{u}_k}\cdot ( \mathbf u-\mathbf u_k )$$
where $\mathbf x_k=\mathbf m(\mathbf u_k)$ is the mean of the gaussian in ray space (can be transformed using the mapping m, the projection matrix) and the Jacobian $ J_{u_k} = \frac{\partial m}{\partial u}(u_k) $

### The gaussian mapping
#### Object to camera coordinates (as detailed in Affine projection of the gauusian)

The reconstruction kernels are initially given in object space, which has coordinates $t =(t_0,t_1,t_2)^T$ . 
* Gaussian reconstruction kernels in object space: $r"_k (t)=G_{V"_k} (t − t_k)$, where $t_k$ are the voxel positions in object space
* Object coordinates are transformed to camera coordinates using an affine mapping $\mathbf u = \phi(t)$, called viewing transformation. It is defined by a matrix $\mathbf W$ and a translation vector $\mathbf d$ as $\phi(t) = Wt + d$. We transform the reconstruction kernels $G_{V"k} (t − t_k)$ to camera space by substituting $t = \phi ^{−1}(u)$ and using Equation 2 : 

$G \mathbf{V"_{k}}(\Phi^{-1}(u) - t_k) = \frac{1}{|\mathbf{W}^{-1}|} G_{V'_k } (\mathbf u - \mathbf u_k) = r'_k(u\mathbf).$

where $u_k = \phi(\mathbf t_k)$ is the center of the Gaussian in camera coordinates and $r′_k(\mathbf u)$ denotes the reconstruction kernel in camera space. According to equation 2, the covariance matrix in camera coordinates $V′_k$ is given by $V′_k = WV"_kW^T$ (exactly as detailed in Affine projection of the gauusian).

#### Camera to ray space
Reminder: the local affine transformation - $$\mathbf{m_{u_k}\mathbf(u)} = \mathbf x_k+\mathbf J_{\mathbf{u}_k}\cdot ( \mathbf u-\mathbf u_k )$$
$\mathbf{m_{u_k}\mathbf(u)}$ transforms from camera to ray, hence $x_k=m(u_k)$ where $x_k$ is the center of the gaussian in ray space. 
since $\mathbf x = m(\mathbf u)$ we get that $ u = \mathbf m^{-1}(\mathbf(x))$ hence: 
$$ u = \mathbf m^{-1}(x) = \mathbf(x - x_k)\cdot \mathbf J^{-1}_{\mathbf u_k} + \mathbf u_k $$
$$ r_k(\mathbf x) = \frac{1}{|\mathbf W^{-1}|}\cdot G_{\mathbf V'_k}(\mathbf{(x - x_k)\cdot J^{-1}_{\mathbf u_k} + \mathbf u_k - \mathbf u_k}) = \\
\frac{1}{|\mathbf W^{-1}||\mathbf J^{-1}|}G_{\mathbf V'_k}(J^{-1}_{\mathbf u_k}\mathbf{(x - x_k)})\\$$

and the new covariance matrix is defined:  $Vk = JV'_{k}$ $J^T = JWV"_kW^TJ^T$.
where V"_k is the original covariance matrix





### Perspective Projection and Jacobian Derivation

In a pinhole camera model, the perspective projection of a 3D point \((X, Y, Z)\) into 2D image coordinates \((x, y)\) can be expressed as:

$$
\begin{bmatrix}
x \\
y \\
1
\end{bmatrix} = \frac{1}{Z} \cdot
\begin{bmatrix}
f_x \cdot X + c_x \cdot Z \\
f_y \cdot Y + c_y \cdot Z \\
Z
\end{bmatrix}
$$

Where:

* $(x, y)$  : are the projected 2D coordinates on the image plane.
* $(X, Y, Z)$  : are the coordinates in the 3D space. 
* $f_x , f_y$  : are the focal lengths in the x and y directions, respectively. 
* $c_x , c_y$  : are the coordinates of the principal point (optical center) in the image. 

### Derivation of the Jacobian

To derive the Jacobian, we compute the partial derivatives of the projected 2D coordinates \((x, y)\) with respect to the 3D coordinates \((X, Y, Z)\).

1. **From the perspective projection equations**:
   - For \(x\):
   $$
   x = \frac{f_x \cdot X + c_x \cdot Z}{Z}
   $$
   - For \(y\):
   $$
   y = \frac{f_y \cdot Y + c_y \cdot Z}{Z}
   $$

2. **Calculating the derivatives**:
   - The derivatives with respect to \(Z\) yield:
     - For \(x\):
     $$
     \frac{\partial x}{\partial X} = \frac{f_x}{Z}, \quad \frac{\partial x}{\partial Y} = 0, \quad \frac{\partial x}{\partial Z} = -\frac{f_x \cdot X}{Z^2}
     $$
     - For \(y\):
     $$
     \frac{\partial y}{\partial X} = 0, \quad \frac{\partial y}{\partial Y} = \frac{f_y}{Z}, \quad \frac{\partial y}{\partial Z} = -\frac{f_y \cdot Y}{Z^2}
     $$

3. **Forming the Jacobian**:
   Using the derivatives calculated above, the Jacobian matrix \(J\) becomes:
   $$
   J = \begin{bmatrix}
   \frac{f_x}{Z} & 0 & -\frac{f_x \cdot X}{Z^2} \\
   0 & \frac{f_y}{Z} & -\frac{f_y \cdot Y}{Z^2}
   \end{bmatrix}
   $$


### 2D Projection 
We calculate the conics coeffitient for each gaussian : $$Ax^2+Bxy+Cy^2 = 0$$, where $A = \frac{\sigma_y^2}{|V|}$, $B = \frac{\sigma_xy}{|V|}$ and  $C = \frac{\sigma_x^2}{|V|}$
we then multiply by the distace between the gaussiam and the pixel. this is the same as: 
$-\frac{1}{2} (\mathbf{x} - \mathbf{p})^T \mathbf{V}^{-1} (\mathbf{x} - \mathbf{p})$ - the exponent of the gaussian

wIncorporating the scaled conic, which represents the 2D Gaussian as an ellipse in the Gaussian equation, we can express the effect of every Gaussian on the pixel's color as follows:

$$
G_{\mathbf{V}}(\mathbf{x}) = \frac{1}{(2\pi)^{3/2} |\mathbf{V}|^{1/2}} e^{-\frac{1}{2} ( Ax^2+Bxy+Cy^2)}
$$

This equation captures how each Gaussian contributes to the pixel's color based on its distance from the pixel and the shape defined by the covariance matrix.

## tiles
To render the image we devide it to tiles of $ 16 X 16 $, for each tile we sort the gaussians according to the distance from the camera. we then sample all gaussians effect per pixel in the tile (using the power equation to calculate the opacity). We sum the gaussians color values and multiply by the opacity (doing alpha blending, each time we update the opacity according to the already calculated gaussians $color = RGB * \alpha *\alpha_{blend})$ were for eacg gaussian we add we update $\alpha_{blend} =  \alpha_{blend}*(1-\alpha) $ (when we reach 0, its opaque). to later do the alpha blend we pick for each tile only the gaussians that intersect with the tile. since gaussians are not bounded, we bound them with 3 $\sigma_x$ and 3 $\sigma_y$ bounding box in each axis. 

